In [10]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.table import Table
import pandas as pd
import astropy.units as u
from astropy.io import fits
from astropy.cosmology import Planck15 as cosmo

In [15]:
corrected_path = '../Rand_Ha/'
table_path = '../tables/'
cat_path = '../CANDLES_data/goods_catalogs/'
total_path = '../total_gal/'
cutout_path = '../CANDLES_data/total_cutouts/'
save_cutout_path = '../CANDLES_data/merger_cutouts/'
seg_path = '../CANDLES_data/goodsn_3dhst_v4.0_f160w/'
Ha_path = '../merger_candidates/'
image_path = '../images/'

# Making Tabels
After going through the GOODS Fields objects and selecting possible merger cadidates by eye, we then must append their ID number, ra, dec, and z values, and what ID they are merging with to ensure they are actually within a range close enough to consider them merging.

In [7]:
Sections = ['GN1', 'GN2', 'GN3', 'GN4', 'GN5', 'GN7', 'GS1', 'GS2', 'GS3', 'GS4', 'GS5', 'ERSPRIME']                 

Merger_dict = {'GN1':[], 'GN2':[], 'GN3':[], 'GN4':[], 'GN5':[], 'GN7':[], 
              'GS1':[], 'GS2':[], 'GS3':[], 'GS4':[], 'GS5':[], 'ERSPRIME':[],
              'GN1_ra':[], 'GN2_ra':[], 'GN3_ra':[], 'GN4_ra':[], 'GN5_ra':[], 'GN7_ra':[],
              'GS1_ra':[], 'GS2_ra':[], 'GS3_ra':[], 'GS4_ra':[], 'GS5_ra':[], 'ERSPRIME_ra':[],
              'GN1_dec':[], 'GN2_dec':[], 'GN3_dec':[], 'GN4_dec':[], 'GN5_dec':[], 'GN7_dec':[],
              'GS1_dec':[], 'GS2_dec':[], 'GS3_dec':[], 'GS4_dec':[], 'GS5_dec':[], 'ERSPRIME_dec':[],
              'GN1_z':[], 'GN2_z':[], 'GN3_z':[], 'GN4_z':[], 'GN5_z':[], 'GN7_z':[],
              'GS1_z':[], 'GS2_z':[], 'GS3_z':[], 'GS4_z':[], 'GS5_z':[], 'ERSPRIME_z':[],
              'GN1_merge':[], 'GN2_merge':[], 'GN3_merge':[], 'GN4_merge':[], 'GN5_merge':[], 'GN7_merge':[], 
              'GS1_merge':[], 'GS2_merge':[], 'GS3_merge':[], 'GS4_merge':[], 'GS5_merge':[], 'ERSPRIME_merge':[]}

for i in Sections:
    Test_Subjects = np.loadtxt(total_path+'{}_total.txt'.format(i)).astype(int)
    
    if i[1] == 'N':
        cat = Table.read(fits.open(cat_path+'goodsn_3dhst.v4.4.zout.fits'), format = 'fits').to_pandas()
    else:
        cat = Table.read(fits.open(cat_path+'goodss_3dhst.v4.4.zout.fits'), format = 'fits').to_pandas()

    for ii in Test_Subjects:
        Merger_dict[i].append(ii)
        Merger_dict['{}_ra'.format(i)].append(cat.query('id == {}'.format(ii)).ra.values[0])
        Merger_dict['{}_dec'.format(i)].append(cat.query('id == {}'.format(ii)).dec.values[0])
        Merger_dict['{}_z'.format(i)].append(cat.query('id == {}'.format(ii)).z500.values[0])

Now we must test if these objects are actually merging. We do this by finding the difference in ra, dec, and z between the main object and the object(s) that we believe it could be merging with. We defined these values to be: <br>
44.3 kiloparsecs for ra and dec. <br>
0.06 for redshift (our z value).

In [11]:
Sections = ['GN1', 'GN2', 'GN3', 'GN4', 'GN5', 'GN7', 'GS1', 'GS2', 'GS3', 'GS4', 'GS5', 'ERSPRIME']                

for i in Sections:   
    if i[1] == 'N':
        cat = Table.read(fits.open(cat_path+'goodsn_3dhst.v4.4.zout.fits'), format = 'fits').to_pandas()
    else:
        cat = Table.read(fits.open(cat_path+'goodss_3dhst.v4.4.zout.fits'), format = 'fits').to_pandas()
    
    for ii in range(len(Merger_dict['{}_ra'.format(i)])):
        ra = Merger_dict['{}_ra'.format(i)][ii]
        dec = Merger_dict['{}_dec'.format(i)][ii]
        z = Merger_dict['{}_z'.format(i)][ii]


        diff_ra = np.abs(ra - cat['ra'].values)
        diff_dec = np.abs(dec - cat['dec'].values)
        diff_z = np.abs(z - cat['z500'].values) / (1 + cat['z500'].values)

        diff_ra_kpc = (1/cosmo.arcsec_per_kpc_proper(z)) * diff_ra*u.deg * 60**2*u.arcsec / u.deg
        diff_dec_kpc = (1/cosmo.arcsec_per_kpc_proper(z)) * diff_dec*u.deg * 60**2*u.arcsec / u.deg
        
        z_pass = cat['id'].values[diff_z < 0.06]
        ra_pass = cat['id'].values[diff_ra_kpc.value < 44.3]
        dec_pass = cat['id'].values[diff_dec_kpc.value < 44.3]

        mergers = []
        
        for RA in ra_pass:
            if RA in dec_pass and RA in z_pass and RA != Merger_dict[i][ii]:
                mergers.append(RA)
        Merger_dict['{}_merge'.format(i)].append(mergers)

/Users/felix/anaconda3/envs/grizli/lib/python3.6/site-packages/astropy/units/quantity.py:477: RuntimeWarning: divide by zero encountered in true_divide
  result = super().__array_ufunc__(function, method, *arrays, **kwargs)


Now any objects that pass the test are appended into a Pandas Dictionary and then downloaded to prevent the code from running again.

In [12]:
#MAKING A PANDAS DICTIONARY
#GN1
GN1_data =  {'ids': pd.Series(Merger_dict['GN1']), 'ra': pd.Series(Merger_dict['GN1_ra']), 
              'dec': pd.Series(Merger_dict['GN1_dec']),'z': pd.Series(Merger_dict['GN1_z']), 
              'mergers': pd.Series(Merger_dict['GN1_merge'])}
GN1_table = pd.DataFrame(GN1_data)
#GN2
GN2_data =  {'ids': pd.Series(Merger_dict['GN2']), 'ra': pd.Series(Merger_dict['GN2_ra']), 
              'dec': pd.Series(Merger_dict['GN2_dec']),'z': pd.Series(Merger_dict['GN2_z']), 
              'mergers': pd.Series(Merger_dict['GN2_merge'])}
GN2_table = pd.DataFrame(GN2_data)
#GN3
GN3_data =  {'ids': pd.Series(Merger_dict['GN3']), 'ra': pd.Series(Merger_dict['GN3_ra']), 
              'dec': pd.Series(Merger_dict['GN3_dec']),'z': pd.Series(Merger_dict['GN3_z']), 
              'mergers': pd.Series(Merger_dict['GN3_merge'])}
GN3_table = pd.DataFrame(GN3_data)
#GN4
GN4_data =  {'ids': pd.Series(Merger_dict['GN4']), 'ra': pd.Series(Merger_dict['GN4_ra']), 
              'dec': pd.Series(Merger_dict['GN4_dec']),'z': pd.Series(Merger_dict['GN4_z']), 
              'mergers': pd.Series(Merger_dict['GN4_merge'])}
GN4_table = pd.DataFrame(GN4_data)
#GN5
GN5_data =  {'ids': pd.Series(Merger_dict['GN5']), 'ra': pd.Series(Merger_dict['GN5_ra']), 
              'dec': pd.Series(Merger_dict['GN5_dec']),'z': pd.Series(Merger_dict['GN5_z']), 
              'mergers': pd.Series(Merger_dict['GN5_merge'])}
GN5_table = pd.DataFrame(GN5_data)
#GN7
GN7_data =  {'ids': pd.Series(Merger_dict['GN7']), 'ra': pd.Series(Merger_dict['GN7_ra']), 
              'dec': pd.Series(Merger_dict['GN7_dec']),'z': pd.Series(Merger_dict['GN7_z']), 
              'mergers': pd.Series(Merger_dict['GN7_merge'])}
GN7_table = pd.DataFrame(GN7_data)
#GS1
GS1_data =  {'ids': pd.Series(Merger_dict['GS1']), 'ra': pd.Series(Merger_dict['GS1_ra']), 
              'dec': pd.Series(Merger_dict['GS1_dec']),'z': pd.Series(Merger_dict['GS1_z']), 
              'mergers': pd.Series(Merger_dict['GS1_merge'])}
GS1_table = pd.DataFrame(GS1_data)
#GS2
GS2_data =  {'ids': pd.Series(Merger_dict['GS2']), 'ra': pd.Series(Merger_dict['GS2_ra']), 
              'dec': pd.Series(Merger_dict['GS2_dec']),'z': pd.Series(Merger_dict['GS2_z']), 
              'mergers': pd.Series(Merger_dict['GS2_merge'])}
GS2_table = pd.DataFrame(GS2_data)
#GS3
GS3_data =  {'ids': pd.Series(Merger_dict['GS3']), 'ra': pd.Series(Merger_dict['GS3_ra']), 
              'dec': pd.Series(Merger_dict['GS3_dec']),'z': pd.Series(Merger_dict['GS3_z']), 
              'mergers': pd.Series(Merger_dict['GS3_merge'])}
GS3_table = pd.DataFrame(GS3_data)
#GS4
GS4_data =  {'ids': pd.Series(Merger_dict['GS4']), 'ra': pd.Series(Merger_dict['GS4_ra']), 
              'dec': pd.Series(Merger_dict['GS4_dec']),'z': pd.Series(Merger_dict['GS4_z']), 
              'mergers': pd.Series(Merger_dict['GS4_merge'])}
GS4_table = pd.DataFrame(GS4_data)
#GS5
GS5_data =  {'ids': pd.Series(Merger_dict['GS5']), 'ra': pd.Series(Merger_dict['GS5_ra']), 
              'dec': pd.Series(Merger_dict['GS5_dec']),'z': pd.Series(Merger_dict['GS5_z']), 
              'mergers': pd.Series(Merger_dict['GS5_merge'])}
GS5_table = pd.DataFrame(GS5_data)
#ERSPRIME
ERSPRIME_data =  {'ids': pd.Series(Merger_dict['ERSPRIME']), 'ra': pd.Series(Merger_dict['ERSPRIME_ra']), 
              'dec': pd.Series(Merger_dict['ERSPRIME_dec']),'z': pd.Series(Merger_dict['ERSPRIME_z']), 
              'mergers': pd.Series(Merger_dict['ERSPRIME_merge'])}
ERSPRIME_table = pd.DataFrame(ERSPRIME_data)

#Uncomment these steps to download the data:
#GN1_table.to_csv(r'/Users/felixmartinez/Research/Merging_Tables/GN1_data.csv')
#GN2_table.to_csv(r'/Users/felixmartinez/Research/Merging_Tables/GN2_data.csv')
#GN3_table.to_csv(r'/Users/felixmartinez/Research/Merging_Tables/GN3_data.csv')
#GN4_table.to_csv(r'/Users/felixmartinez/Research/Merging_Tables/GN4_data.csv')
#GN5_table.to_csv(r'/Users/felixmartinez/Research/Merging_Tables/GN5_data.csv')
#GN7_table.to_csv(r'/Users/felixmartinez/Research/Merging_Tables/GN7_data.csv')
#GS1_table.to_csv(r'/Users/felixmartinez/Research/Merging_Tables/GS1_data.csv')
#GS2_table.to_csv(r'/Users/felixmartinez/Research/Merging_Tables/GS2_data.csv')
#GS3_table.to_csv(r'/Users/felixmartinez/Research/Merging_Tables/GS3_data.csv')
#GS4_table.to_csv(r'/Users/felixmartinez/Research/Merging_Tables/GS4_data.csv')
#GS5_table.to_csv(r'/Users/felixmartinez/Research/Merging_Tables/GS5_data.csv')
#ERSPRIME_table.to_csv(r'/Users/felixmartinez/Research/Merging_Tables/ERSPRIME_data.csv')

# Filtering Non Mergers & Making the Lists
Now we must check the data. We do this by opening up all the dictionaries we created and removing any z values that were recorded as zero, as this flags every object as a merger. Once we filter out all z = 0 values, we create a new dataframe.

In [13]:
#First we have to make arrays for the galaxy mergers, ids, ra, dec, and z values for each field.
merger_cat = {'GN1':[], 'GN2':[], 'GN3':[], 'GN4':[], 'GN5':[], 'GN7':[], 
           'GS1':[], 'GS2':[], 'GS3':[], 'GS4':[], 'GS5':[], 'ERSPRIME':[]}
mergers = {'GN1':[], 'GN2':[], 'GN3':[], 'GN4':[], 'GN5':[], 'GN7':[], 
           'GS1':[], 'GS2':[], 'GS3':[], 'GS4':[], 'GS5':[], 'ERSPRIME':[]}
ids = {'GN1':[], 'GN2':[], 'GN3':[], 'GN4':[], 'GN5':[], 'GN7':[], 
       'GS1':[], 'GS2':[], 'GS3':[], 'GS4':[], 'GS5':[], 'ERSPRIME':[]}
ra = {'GN1':[], 'GN2':[], 'GN3':[], 'GN4':[], 'GN5':[], 'GN7':[], 
      'GS1':[], 'GS2':[], 'GS3':[], 'GS4':[], 'GS5':[], 'ERSPRIME':[]}
dec = {'GN1':[], 'GN2':[], 'GN3':[], 'GN4':[], 'GN5':[], 'GN7':[], 
       'GS1':[], 'GS2':[], 'GS3':[], 'GS4':[], 'GS5':[], 'ERSPRIME':[]}
z = {'GN1':[], 'GN2':[], 'GN3':[], 'GN4':[], 'GN5':[], 'GN7':[], 
     'GS1':[], 'GS2':[], 'GS3':[], 'GS4':[], 'GS5':[], 'ERSPRIME':[]}

tables = [GN1_table, GN2_table, GN3_table, GN4_table, GN5_table, GN7_table, 
          GS1_table, GS2_table, GS3_table, GS4_table, GS5_table, ERSPRIME_table]
fields = ['GN1', 'GN2', 'GN3', 'GN4', 'GN5', 'GN7', 
          'GS1', 'GS2', 'GS3', 'GS4', 'GS5', 'ERSPRIME']

for t in range(len(tables)):
    Merging_Data = []
    field = fields[t]
    table = tables[t]
    for i in range(len(table)):
        if (table.z[i] == 0) or (len(table.at[i,'mergers']) == 0) or (table.z[i] < 1) or (table.z[i] > 2.2):
            pass;
        else:
            mergers[field].append(table.mergers.values[i])
            ids[field].append(table.ids.values[i])
            ra[field].append(table.ra.values[i])
            dec[field].append(table.dec.values[i])
            z[field].append(table.z.values[i])
    print('Field: '+ field + ' completed.')
        
        
    Merging_Data = {'ids': pd.Series(ids),'ra': pd.Series(ra),'dec': pd.Series(dec),
                    'z': pd.Series(z), 'mergers': pd.Series(mergers)}
    merger_cat = pd.DataFrame(Merging_Data).T
#merger_cat.to_pickle(table_path+'merger_cat.plk')
print('Merger Catalog saved successfully as pickle.')

Field: GN1 completed.
Field: GN2 completed.
Field: GN3 completed.
Field: GN4 completed.
Field: GN5 completed.
Field: GN7 completed.
Field: GS1 completed.
Field: GS2 completed.
Field: GS3 completed.
Field: GS4 completed.
Field: GS5 completed.
Field: ERSPRIME completed.
Merger Catalog saved successfully as pickle.


In [14]:
merger_cat['GN1']

ids        [37786, 37787, 36203, 37053, 37395, 38099, 373...
ra         [189.281986, 189.325931, 189.312368, 189.28273...
dec        [62.363756, 62.363382, 62.335707, 62.346241, 6...
z          [1.4972765422373637, 1.3583397425048862, 1.748...
mergers    [[37738, 37836], [37788], [36162], [37095, 371...
Name: GN1, dtype: object